#### Internal Working of Threads

- Python program starts with MainThread.

- When we create Thread(...) and call .start(), Python asks the OS scheduler to allocate time for this thread.

- OS decides when to context-switch between threads.

- Threads share same memory space, so they can read/write the same variables.

- Python’s GIL ensures only one thread runs Python code at once → but if one is waiting (sleep, I/O), another gets CPU.

In [ ]:
import threading
import time

def print_numbers():
    '''Thread function to print Numbers'''
    for i in range(4):
        print(f"[{threading.current_thread().name}] Number: {i}")
        time.sleep(1)
        
def print_letters():
    '''Thread function to print Letters'''
    for ch in "ABCD":
        print(f"[{threading.current_thread().name}] Letter: {ch}")
        time.sleep(1)
        
t1 = threading.Thread(target=print_numbers, name="Getting Numbers")
t2 = threading.Thread(target=print_letters, name="Getting Letters")

t1.start()
t2.start()

t1.join()
t2.join()

print("Program Executed!...")

In [ ]:
import multiprocessing
import time

def print_numbers():
    '''Process function to print Numbers'''
    for i in range(4):
        print(f"[{multiprocessing.current_process().name}] Number: {i}")
        time.sleep(1)
        
def print_letters():
    '''Process function to print Letters'''
    for ch in "ABCD":
        print(f"[{multiprocessing.current_process().name}] Letter: {ch}")
        time.sleep(1)
        
p1 = multiprocessing.Process(target=print_numbers, name="Numbers Pool")
p2 = multiprocessing.Process(target=print_letters, name="Letter Pool")

p1.start()
p2.start()

p1.join()
p2.join()

Global Interpreter Lock does not prevent races — it simply prevents multiple threads from executing Python bytecode simultaneously. But it does not make compound operations atomic; the scheduler can switch between bytecode instructions.

##### Multiprocessing

- Multiprocessing refers to running multiple processes simultaneously, where each process has its own memory space and runs independently.
- Each process is a heavyweight execution unit, including its own Python interpreter and memory.
- Multiprocessing bypasses Python’s Global Interpreter Lock (GIL), since each process runs in a separate interpreter.
- This enables true parallelism on multi-core CPUs.


#### Internal Working of Multiprocessing

1. Python program starts with **MainProcess**.
2. When `.start()` is called:
    - A new Python interpreter is launched.
    - The target function runs in its own memory space.
3. Processes don’t share memory directly.
4. Communication between processes requires IPC (**Inter-Process Communication**) tools like:
    - **Queue**: Safe for sending data between processes.
    - **Pipe**: Connects two processes.
    - **Manager**: Shared objects.


In [ ]:
import multiprocessing
import time
import math
import sys

sys.set_int_max_str_digits(1000000)


def calc_factorial(n):
    print(f"[{multiprocessing.current_process().name}] Calculating Factorial of {n}")
    result = math.factorial(n)
    print(f"Factorial of {n}! is {result}")


if __name__ == "__main__":
    numbers = [20000, 7500, 2000, 4000, 8000, 7000, 11000, 330000, 90000]

    start_time = time.time()
    processes = []

    for n in numbers:
        process = multiprocessing.Process(target=calc_factorial, args=(n,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    print(processes)
    print(f"Time Taken: {time.time() - start_time} Seconds")


In [ ]:
import threading
import time
import math
import sys

sys.set_int_max_str_digits(1000000)


def calc_factorial(n):
    print(f"[{threading.current_thread().name}] Calculating Factorial of {n}")
    result = math.factorial(n)
    print(f"Factorial of {n}! is {result}")


if __name__ == "__main__":
    numbers = [20000, 7500, 2000, 4000, 8000, 7000, 11000, 330000, 90000]

    start_time = time.time()
    threads = []

    for n in numbers:
        thread = threading.Thread(target=calc_factorial, args=(n,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print(threads)
    print(f"Time Taken: {time.time() - start_time} Seconds")


In [ ]:
from multiprocessing import Pool

def square(n):
    return n * n

if __name__ == "__main__":
    numbers = [2, 34, 85, 12, 8]
    with Pool(processes=3) as pool:
        result = pool.map(square, numbers)
    print("Squares: ",result)

A **race condition** occurs when two or more threads or processes access and modify shared data (or resources) at the same time, and the final outcome depends on the timing or order of their execution. This non-deterministic behavior can lead to bugs that appear only occasionally.

**Analogy:**  
Imagine two cashiers updating the same balance on a paper ledger simultaneously. If both read the old balance, add their amounts, and write back, one update can be lost depending on who writes last.

**Why it matters:**  
Race condition bugs are unpredictable, difficult to reproduce, and can corrupt data or cause security and consistency failures.

In [ ]:
import threading
import time 

counter = 0
lock = threading.Lock()
def worker(n):
    for _ in range(n):
        global counter
        with lock:
            current = counter
            time.sleep(0)
            counter = current + 1

if __name__ == "__main__":
    N_THREADS = 10
    INCREMENTS = 1000

    threads = []
    for i in range(N_THREADS):
        t = threading.Thread(target=worker, args=(INCREMENTS,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    print("Expected:", N_THREADS * INCREMENTS)
    print("Actual:  ", counter)


In [ ]:

import threading
import time
import dis

counter = 0

def worker(n):
    global counter
    for _ in range(n):
        current = counter
        # time.sleep(0)  
        counter = current + 1
        
dis.dis(worker)

if __name__ == "__main__":
    N_THREADS = 10
    INCREMENTS = 100

    threads = []
    for i in range(N_THREADS):
        t = threading.Thread(target=worker, args=(INCREMENTS,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    print("Expected:", N_THREADS * INCREMENTS)
    print("Actual:  ", counter)


lock = threading.Lock() creates a primitive mutex.

with lock: does lock.acquire() then try: ... finally: lock.release().

In [ ]:
import threading

rlock = threading.RLock()

shared = []

def inner_inside_inner():
    with rlock:
        shared.append("Nested Inner Called")

def inner():
    with rlock:
        shared.append("Inner Called")
        inner_inside_inner()
        
def outer():
    with rlock:
        shared.append("Outer Starting")
        inner()
        shared.append("Outer Ending")

if __name__ == "__main__":
    t1 = threading.Thread(target=outer)
    t1.start()
    t1.join()
    print(shared)

RLock tracks ownership and acquisition count, so re-entry is allowed for the owning thread.

In [ ]:
import threading
import time

lock_a = threading.Lock()
lock_b = threading.Lock()

def thread1():
    with lock_a:
        time.sleep(0)
        with lock_b:
            print("thread1 acquire both")

def thread2():
    with lock_a:
        time.sleep(0)
        with lock_b:
            print("thread2 acquire both")
            
if __name__ == "__main__":
    devlop = threading.Thread(target=thread1)
    lopment = threading.Thread(target=thread2)
    devlop.start()
    lopment.start()
    devlop.join()
    lopment.join()
    print("Development")

**Deadlock Example:**  
When two threads each hold a lock and wait for the other's lock, neither can proceed—this is a classic deadlock.  
For instance: `devlop` (thread1) acquires `lock_a` and waits for `lock_b`, while `lopment` (thread2) acquires `lock_b` and waits for `lock_a`. Both are stuck.

**How to Avoid Deadlocks:**

- **Consistent Lock Ordering:** Always acquire locks in the same global order across all threads.
- **Timeouts:** Use `lock.acquire(timeout=...)` and retry or back off if acquisition fails.
- **Simplify Locking:** Prefer a single lock or coarser-grained locks to reduce complexity.
- **High-Level Synchronization:** Use constructs like `Queue`, `Condition`, or `Manager` for safer coordination.

Deadlocks are subtle and can halt your program—design your locking strategy thoughtfully!

##### Semaphore — limit concurrency (counting lock)

Purpose: control number of concurrent threads accessing a resource (e.g., limit concurrent downloads to 5).

In [ ]:
import threading
import random
import time

sem = threading.Semaphore(2)

def employee(member):
    print(f"{member} waiting for the system.....")
    with sem:
        print(f"{member} aquired the system\n")
        time.sleep(random.uniform(0.5, 1.5))
        print(f"{member} release the system!!!")
    
if __name__ == "__main__":
    members = ["Rishikesh", "Alberto", "Harry", "Ron", "Aniket", "Atharv", "William"]
    employees = [threading.Thread(target=employee, args=(member,)) for member in members]
    for thread in employees: thread.start()
    for thread in employees: thread.join()
    print("You did it")    

In [ ]:

def get_forwarded_result_of(text):
    modified_text = []
    for ch in text:
        if(ch != " "):
            modified_text.append(chr(ord(ch)+1))
        else:
            modified_text.append(ch)
    return "".join(modified_text)

print(get_forwarded_result_of("Ibsf Lsjtiob Ibsf Sbnb"))
print(get_forwarded_result_of("ab ef hi"))

- Data Structures: lists, tuples, sets, dictionaries  
- File Handling: reading/writing CSV, JSON, text, binary files  
- Pandas: dataframes, filtering, grouping, merging, aggregation  
- NumPy: arrays, vectorized operations, broadcasting  
- Data Cleaning: handling missing values, duplicates, outliers  
- Data Visualization: matplotlib, seaborn, plotly  
- SQL Integration: querying databases with sqlite3, SQLAlchemy, pandas  
- Regular Expressions: pattern matching, text extraction  
- ETL Pipelines: extract-transform-load workflows  
- APIs: consuming REST APIs, requests, authentication  
- Parallelism: multiprocessing, threading, concurrent.futures  
- Logging & Error Handling: robust scripts, debugging  
- Automation: scheduling jobs, batch processing  
- Cloud Integration: AWS S3, GCP, Azure SDKs  
- Unit Testing: pytest, unittest for data pipelines  
- Serialization: pickle, joblib, saving models and data  
- Data Modeling: basic statistics, feature engineering  
- Version Control: git, collaborative workflows

In [ ]:
import threading
import time

event = threading.Event()

def waiter(n):
    print(f"Waiter {n} waiting for the event")
    event.wait()
    print(f"Waiter {n} receive the event")
    
def setter():
    print("\nSetter: setting event:\n")
    time.sleep(2)
    event.set()
    
if __name__ == "__main__":
    threads = [threading.Thread(target=waiter, args=(i, )) for i in range(1, 7)]
    for t in threads: t.start()
    s = threading.Thread(target=setter)
    s.start()
    for t in threads: t.join()
    s.join()

In [ ]:
import threading
lock = threading.RLock()
def func():
    with lock:
        print("Got it!")
threading.Thread(target=func).start()

In [ ]:
import threading
class Counter:
    def __init__(self):
        self.x = 0
        print(self.x)
        
    def increment(self):
        self.x += 1
        print(self.x)

c = Counter()
threading.Thread(target=c.increment).start()  
threading.Thread(target=Counter.increment, args=(c,)).start()  

In [ ]:
import threading
def worker():
    import time  
    print("Working...")
    time.sleep(1)
    print("Done")

for _ in range(3):
    threading.Thread(target=worker).start()

In [ ]:
import multiprocessing

def rancho_introduces_himself():
    print(f"Aal izz well! I am: {multiprocessing.current_process().name}")

rancho_thread = multiprocessing.Process(target=rancho_introduces_himself)

print(f"When Raju and Farhan meets: {rancho_thread.name}")

rancho_thread.name = "Phunsukh_Wangdu"

rancho_thread.start()

When Raju and Farhan meets: Process-3
Aal izz well! I am: Phunsukh_Wangdu

## English Description of Producer-Consumer Synchronization Program

**Purpose:**
This program demonstrates a classic multi-threading pattern called the "Producer-Consumer Problem" where one thread creates items and another thread consumes them, with proper synchronization to prevent conflicts.

**Main Components:**

1. **Shared Buffer:** A queue (deque) that can hold a maximum of 5 items at any time
2. **Synchronization Tool:** A condition variable that helps threads communicate and wait for each other
3. **Two Types of Workers:** Producer thread (creates items) and Consumer thread (removes items)

**How the Producer Function Works:**
- Takes a name and number of items to produce as inputs
- Runs in a loop to create the specified number of items
- Each item is labeled with the producer's name and a sequence number (like "P-0", "P-1")
- Before adding an item to the buffer:
  - Acquires a lock to safely access the shared buffer
  - Checks if the buffer is full (5 items max)
  - If full, waits until a consumer removes something
  - If space available, adds the new item to the buffer
  - Prints a message showing what was produced and current buffer size
  - Notifies all waiting consumer threads that a new item is available
- Sleeps for a random time between 0.1 and 0.4 seconds before making the next item

**How the Consumer Function Works:**
- Takes a name and number of items to consume as inputs
- Runs in a loop to consume the specified number of items
- Before taking an item from the buffer:
  - Acquires a lock to safely access the shared buffer
  - Checks if the buffer is empty
  - If empty, waits until a producer adds something
  - If items available, removes the oldest item from the front of the buffer
  - Prints a message showing what was consumed and current buffer size
  - Notifies all waiting producer threads that space is now available
- Sleeps for a random time between 0.1 and 0.4 seconds before consuming the next item

**Program Execution Flow:**
1. Creates two separate threads - one producer named "P" and one consumer named "C"
2. Both threads are set to handle 10 items each
3. Starts both threads running simultaneously
4. The main program waits for both threads to complete their work
5. Prints "done" when everything is finished

**Key Synchronization Rules:**
- Only one thread can access the buffer at a time (mutual exclusion)
- Producers must wait when buffer is full
- Consumers must wait when buffer is empty
- Threads notify each other when conditions change (item added/removed)
- Random sleep times simulate real-world processing delays

**Note:** There appears to be a syntax error in the original code where `n*items` should likely be `n_items` in the consumer function's range statement.

A Condition allows threads to wait for some state change, combined with a lock.

In [6]:
import threading
import random
import time 
from collections import deque

buffer = deque()
MAX_ITEMS = 5
condition = threading.Condition()

def producer(name, n_items):
    for i in range(n_items):
        item = f"{name}-{i}"
        with condition:
            while len(buffer) >= MAX_ITEMS:
                condition.wait()
            buffer.append(item)
            print(f"Producer {name} produces {item} (size={len(buffer)})")
            condition.notify_all()
        time.sleep(random.uniform(0.1, 1))
        
def consumer(name, n_items):
    for _ in range(n_items):
        with condition:
            while not buffer:
                condition.wait()
            item = buffer.popleft()
            print(f"consumer {name} consumes {item} (size={len(buffer)})")
            condition.notify_all()
        time.sleep(random.uniform(0.1, 0.2))
        
if __name__ == "__main__":
    prod = threading.Thread(target=producer, args=("Amazon", 10))
    cons = threading.Thread(target=consumer, args=("Customer", 10))
    prod.start() , cons.start()
    prod.join(), cons.join()
    print("All Done")
        
        

Producer Amazon produces Amazon-0 (size=1)
consumer Customer consumes Amazon-0 (size=0)
Producer Amazon produces Amazon-1 (size=1)
consumer Customer consumes Amazon-1 (size=0)
Producer Amazon produces Amazon-2 (size=1)
consumer Customer consumes Amazon-2 (size=0)
Producer Amazon produces Amazon-3 (size=1)
consumer Customer consumes Amazon-3 (size=0)
Producer Amazon produces Amazon-4 (size=1)
consumer Customer consumes Amazon-4 (size=0)
Producer Amazon produces Amazon-5 (size=1)
consumer Customer consumes Amazon-5 (size=0)
Producer Amazon produces Amazon-6 (size=1)
consumer Customer consumes Amazon-6 (size=0)
Producer Amazon produces Amazon-7 (size=1)
consumer Customer consumes Amazon-7 (size=0)
Producer Amazon produces Amazon-8 (size=1)
consumer Customer consumes Amazon-8 (size=0)
Producer Amazon produces Amazon-9 (size=1)
consumer Customer consumes Amazon-9 (size=0)
All Done


In [21]:
import threading
import random
import time 
import queue

buffer = queue.Queue(maxsize=5)

def producer(name, n_items):
    for i in range(n_items):
        item = f"{name}-{i}"
        buffer.put(item)    
        print(f"Producer {name} produces {item} (size={buffer.qsize()})")
        time.sleep(random.uniform(0.1, 1))  
        
def consumer(name, n_items):
    for _ in range(n_items):
        item = buffer.get()
        print(f"consumer {name} consumes {item} (size={buffer.qsize()})")
        buffer.task_done()
    time.sleep(random.uniform(0.1, 0.2))
        
if __name__ == "__main__":
    prod = threading.Thread(target=producer, args=("Amazon", 10))
    cons = threading.Thread(target=consumer, args=("Customer", 10))
    prod.start() , cons.start()
    prod.join(), cons.join()
    print("All Done")
        
        

Producer Amazon produces Amazon-0 (size=1)
consumer Customer consumes Amazon-0 (size=0)
Producer Amazon produces Amazon-1 (size=1)
consumer Customer consumes Amazon-1 (size=0)
Producer Amazon produces Amazon-2 (size=1)consumer Customer consumes Amazon-2 (size=0)

Producer Amazon produces Amazon-3 (size=1)
consumer Customer consumes Amazon-3 (size=0)
Producer Amazon produces Amazon-4 (size=1)
consumer Customer consumes Amazon-4 (size=0)
Producer Amazon produces Amazon-5 (size=1)
consumer Customer consumes Amazon-5 (size=0)
Producer Amazon produces Amazon-6 (size=1)consumer Customer consumes Amazon-6 (size=0)

Producer Amazon produces Amazon-7 (size=1)
consumer Customer consumes Amazon-7 (size=0)
Producer Amazon produces Amazon-8 (size=1)
consumer Customer consumes Amazon-8 (size=0)
Producer Amazon produces Amazon-9 (size=1)consumer Customer consumes Amazon-9 (size=0)

All Done
